In [167]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/multimodal-memes/Multimodal_dataset_assignment3/reference.csv
/kaggle/input/multimodal-memes/Multimodal_dataset_assignment3/labels_pd_pickle
/kaggle/input/multimodal-memes/Multimodal_dataset_assignment3/labels.xlsx
/kaggle/input/multimodal-memes/Multimodal_dataset_assignment3/reference.xlsx
/kaggle/input/multimodal-memes/Multimodal_dataset_assignment3/labels.csv
/kaggle/input/multimodal-memes/Multimodal_dataset_assignment3/reference_df_pickle
/kaggle/input/multimodal-memes/Multimodal_dataset_assignment3/images/image_5576.jpg
/kaggle/input/multimodal-memes/Multimodal_dataset_assignment3/images/image_6267.jpg
/kaggle/input/multimodal-memes/Multimodal_dataset_assignment3/images/image_4213.png
/kaggle/input/multimodal-memes/Multimodal_dataset_assignment3/images/image_747.jpg
/kaggle/input/multimodal-memes/Multimodal_dataset_assignment3/images/image_5775.jpg
/kaggle/input/multimodal-memes/Multimodal_dataset_assignment3/images/image_561.jpg
/kaggle/input/multimodal-memes/Multim

In [168]:
df=pd.read_csv("/kaggle/input/multimodal-memes/Multimodal_dataset_assignment3/labels.csv");
df

,Unnamed: 0,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive
1,1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive
2,2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive
3,3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive
4,4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral
...,...,...,...,...,...,...,...,...,...
6987,6987,image_6988.jpg,Tuesday is Mardi Gras Wednesday is Valentine's...,Tuesday is Mardi Gras Wednesday is Valentine's...,very_funny,twisted_meaning,very_offensive,motivational,neutral
6988,6988,image_6989.jpg,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,funny,twisted_meaning,not_offensive,not_motivational,neutral
6989,6989,image_6990.png,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,funny,general,slight,not_motivational,positive
6990,6990,image_6991.jpg,When I VERY have time is a fantasy No one has ...,When I have time is a fantasy. no one has time...,not_funny,twisted_meaning,not_offensive,motivational,very_positive


In [169]:
unique_hums = df['humour'].apply(lambda x: x).unique()
unique_hums

array(['hilarious', 'not_funny', 'very_funny', 'funny'], dtype=object)

In [170]:
unique_sarc = df['sarcasm'].apply(lambda x: x).unique()
unique_sarc

array(['general', 'not_sarcastic', 'twisted_meaning', 'very_twisted'],
      dtype=object)

In [171]:
unique_off = df['offensive'].apply(lambda x: x).unique()
unique_off

array(['not_offensive', 'very_offensive', 'slight', 'hateful_offensive'],
      dtype=object)

In [172]:
unique_mot = df['motivational'].apply(lambda x: x).unique()
unique_mot

array(['not_motivational', 'motivational'], dtype=object)

In [173]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6992 entries, 0 to 6991
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         6992 non-null   int64 
 1   image_name         6992 non-null   object
 2   text_ocr           6831 non-null   object
 3   text_corrected     6987 non-null   object
 4   humour             6992 non-null   object
 5   sarcasm            6992 non-null   object
 6   offensive          6992 non-null   object
 7   motivational       6992 non-null   object
 8   overall_sentiment  6992 non-null   object
dtypes: int64(1), object(8)
memory usage: 491.8+ KB


In [ ]:
from transformers import AutoTokenizer
model_ckpt = "bert-base-uncased" 
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [175]:
df['text_corrected'] = df['text_corrected'].astype(str)
tokens=tokenizer(df['text_corrected'].tolist(), truncation=True, max_length=512,padding=True, return_tensors="pt")
df['tokens']=list(tokens['input_ids'])

In [176]:
df['tokens']

0       [tensor(101), tensor(2298), tensor(2045), tens...
1       [tensor(101), tensor(1996), tensor(2190), tens...
2       [tensor(101), tensor(3520), tensor(14296), ten...
3       [tensor(101), tensor(2184), tensor(2095), tens...
4       [tensor(101), tensor(2184), tensor(2095), tens...
                              ...                        
6987    [tensor(101), tensor(9857), tensor(2003), tens...
6988    [tensor(101), tensor(2442), tensor(3422), tens...
6989    [tensor(101), tensor(2625), tensor(2062), tens...
6990    [tensor(101), tensor(2043), tensor(1045), tens...
6991    [tensor(101), tensor(1996), tensor(3225), tens...
Name: tokens, Length: 6992, dtype: object

In [177]:
import os
df['image_path'] = df['image_name'].apply(lambda x: os.path.join('/kaggle/input/multimodal-memes/Multimodal_dataset_assignment3/images/', x))

In [178]:
df['image_path']

0       /kaggle/input/multimodal-memes/Multimodal_data...
1       /kaggle/input/multimodal-memes/Multimodal_data...
2       /kaggle/input/multimodal-memes/Multimodal_data...
3       /kaggle/input/multimodal-memes/Multimodal_data...
4       /kaggle/input/multimodal-memes/Multimodal_data...
                              ...                        
6987    /kaggle/input/multimodal-memes/Multimodal_data...
6988    /kaggle/input/multimodal-memes/Multimodal_data...
6989    /kaggle/input/multimodal-memes/Multimodal_data...
6990    /kaggle/input/multimodal-memes/Multimodal_data...
6991    /kaggle/input/multimodal-memes/Multimodal_data...
Name: image_path, Length: 6992, dtype: object

In [179]:
from transformers import ViTImageProcessor
from PIL import Image, ImageFile

# Allow truncated images to load
ImageFile.LOAD_TRUNCATED_IMAGES = True

feature_extractor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

def process_image(image_path):
    try:
        img = Image.open(image_path)
        if img.mode == "P":
            img = img.convert("RGBA")

        if img.mode == "RGBA":
            img = img.convert("RGB")
        img=img.convert("RGB")
        img = img.resize((128,128), Image.LANCZOS)
        return feature_extractor(images=img, return_tensors="pt")["pixel_values"].squeeze(0)
    
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None  # Return None if there's an error

df['image_tokens'] = df['image_path'].apply(process_image)


In [180]:
df['image_tokens'] = df['image_tokens'].apply(lambda x: x.view(-1) if x is not None else None)

In [181]:
from sklearn.decomposition import PCA

image_tokens_array = np.vstack(df['image_tokens'].values)
pca = PCA(n_components=500)
image_tokens_pca = pca.fit_transform(image_tokens_array)
df['image_tokens'] = list(image_tokens_pca)

In [182]:
import torch
import numpy as np
df['concat_tokens'] = df.apply(lambda row: np.concatenate((row['tokens'], row['image_tokens'])), axis=1)
df['concat_tokens']

0       [101.0, 2298.0, 2045.0, 2026.0, 2767.0, 2422.0...
1       [101.0, 1996.0, 2190.0, 1997.0, 1001.0, 2184.0...
2       [101.0, 3520.0, 14296.0, 1030.0, 6167.0, 8091....
3       [101.0, 2184.0, 2095.0, 4119.0, 1011.0, 4086.0...
4       [101.0, 2184.0, 2095.0, 4119.0, 2007.0, 2053.0...
                              ...                        
6987    [101.0, 9857.0, 2003.0, 9388.0, 4305.0, 24665....
6988    [101.0, 2442.0, 3422.0, 5691.0, 1997.0, 2418.0...
6989    [101.0, 2625.0, 2062.0, 3331.0, 4041.0, 14904....
6990    [101.0, 2043.0, 1045.0, 2031.0, 2051.0, 2003.0...
6991    [101.0, 1996.0, 3225.0, 2391.0, 2005.0, 2296.0...
Name: concat_tokens, Length: 6992, dtype: object

In [183]:
len(df['concat_tokens'][0])

761

In [184]:
x = np.array(df['concat_tokens'].tolist())
print(x.nbytes / (1024 ** 2), "MB") 

40.5953369140625 MB


In [185]:
unique_sizes = df['concat_tokens'].apply(lambda x: x.shape[0]).unique()
unique_sizes

array([761])

In [186]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

x=np.array(df['concat_tokens'].tolist())
y_hum=df['humour'].map({'hilarious':1, 'not_funny':0, 'very_funny':1, 'funny':1})
y_sarc=df['sarcasm'].map({'general':1, 'not_sarcastic':0, 'twisted_meaning':1, 'very_twisted':1})
y_off=df['offensive'].map({'not_offensive':0, 'very_offensive':1, 'slight':1, 'hateful_offensive':1})
y_mot=df['motivational'].map({'not_motivational':0, 'motivational':1})

x_train_hum, x_test_hum, y_train_hum, y_test_hum = train_test_split(x, y_hum, test_size=0.2, random_state=42)
x_train_sarc, x_test_sarc, y_train_sarc, y_test_sarc = train_test_split(x, y_sarc, test_size=0.2, random_state=42)
x_train_off, x_test_off, y_train_off, y_test_off = train_test_split(x, y_off, test_size=0.2, random_state=42)
x_train_mot, x_test_mot, y_train_mot, y_test_mot = train_test_split(x, y_mot, test_size=0.2, random_state=42)

In [187]:
x.shape

(6992, 761)

HUMOUR

In [ ]:
model_hum = models.Sequential([
    layers.Dense(64, activation="relu", input_shape=(x.shape[1],)), 
    layers.Dropout(0.3),  
    layers.Dense(32, activation="relu"),
    layers.Dense(3,activation="softmax")
])

model_hum.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [189]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_path = "best_model_hum.keras"

checkpoint_callback = ModelCheckpoint(
    checkpoint_path,
    monitor="val_accuracy",
    save_best_only=True,
    mode="max",
    verbose=1
)

history = model_hum.fit(
    x_train_hum, y_train_hum,
    epochs=20,
    batch_size=32,
    callbacks=[checkpoint_callback]
)
model_hum.save("Final_model_hum.keras")

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:635: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


175/175 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.6257 - loss: 239.9614
Epoch 2/20
118/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6352 - loss: 51.2169

/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_accuracy available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6397 - loss: 48.0643
Epoch 3/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6361 - loss: 19.3061
Epoch 4/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6466 - loss: 9.3993
Epoch 5/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6554 - loss: 5.4331
Epoch 6/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6533 - loss: 4.0594
Epoch 7/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6559 - loss: 2.8212
Epoch 8/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6494 - loss: 2.3813
Epoch 9/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6809 - loss: 1.4767
Epoch 10/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6792 - loss: 1.3298
Epoch 11/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6795 - loss: 1.1510
Epoch 12/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6893 - loss: 1.2146
Epoch 13/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accur

In [ ]:
y_pred_hum = np.argmax(model_hum.predict(x_test_hum), axis=1)

accuracy = accuracy_score(y_test_hum, y_pred_hum)
f1_hum = f1_score(y_test_hum, y_pred_hum, average="macro") 
print("Humour")
print(f"Predicted Labels: {y_pred_hum}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Macro F1 Score: {f1_hum:.4f}")

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Humour
Predicted Labels: [1 1 1 ... 1 1 0]
Accuracy: 0.6562
Macro F1 Score: 0.4928


SARCASM

In [ ]:
model_sarc = models.Sequential([
    layers.Dense(64, activation="relu", input_shape=(x.shape[1],)), 
    layers.Dropout(0.3), 
    layers.Dense(32, activation="relu"),
    layers.Dense(3,activation="softmax") 
])

model_sarc.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [192]:
checkpoint_path = "best_model_sarc.keras"

checkpoint_callback = ModelCheckpoint(
    checkpoint_path,
    monitor="val_accuracy",
    save_best_only=True,
    mode="max",
    verbose=1
)

history = model_sarc.fit(
    x_train_sarc, y_train_sarc,
    epochs=20,
    batch_size=32,
    callbacks=[checkpoint_callback]
)
model_sarc.save("Final_model_sarc.keras")

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:635: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


175/175 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.6485 - loss: 178.4521
Epoch 2/20
119/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6435 - loss: 42.0436

/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_accuracy available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6449 - loss: 39.6564
Epoch 3/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6463 - loss: 17.9693
Epoch 4/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6493 - loss: 8.6267
Epoch 5/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6755 - loss: 5.8873
Epoch 6/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6654 - loss: 2.9952
Epoch 7/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6661 - loss: 2.1887
Epoch 8/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6717 - loss: 1.6985
Epoch 9/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6844 - loss: 1.5408
Epoch 10/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6977 - loss: 1.2676
Epoch 11/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6956 - loss: 1.0828
Epoch 12/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7035 - loss: 0.9056
Epoch 13/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accur

In [ ]:
y_pred_sarc = np.argmax(model_sarc.predict(x_test_sarc), axis=1)

accuracy = accuracy_score(y_test_sarc, y_pred_sarc)
f1_sarc = f1_score(y_test_sarc, y_pred_sarc, average="macro")  
print("Sarcasm")
print(f"Predicted Labels: {y_pred_sarc}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Macro F1 Score: {f1_sarc:.4f}")

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Sarcasm
Predicted Labels: [1 1 1 ... 1 1 1]
Accuracy: 0.7791
Macro F1 Score: 0.4379


OFFENSIVE

In [ ]:
model_off = models.Sequential([
    layers.Dense(64, activation="relu", input_shape=(x.shape[1],)),  
    layers.Dropout(0.3),  
    layers.Dense(32, activation="relu"),
    layers.Dense(3,activation="softmax")  
])

model_off.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [195]:
checkpoint_path = "best_model_off.keras"

checkpoint_callback = ModelCheckpoint(
    checkpoint_path,
    monitor="val_accuracy",
    save_best_only=True,
    mode="max",
    verbose=1
)

history = model_off.fit(
    x_train_off, y_train_off,
    epochs=20,
    batch_size=32,
    callbacks=[checkpoint_callback]
)
model_off.save("Final_model_off.keras")

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:635: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


175/175 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.5060 - loss: 315.8194
Epoch 2/20
117/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5629 - loss: 69.8497

/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_accuracy available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5545 - loss: 67.1597
Epoch 3/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5328 - loss: 29.3263
Epoch 4/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5507 - loss: 16.0558
Epoch 5/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5448 - loss: 9.0718
Epoch 6/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5329 - loss: 5.0694
Epoch 7/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5085 - loss: 3.9544
Epoch 8/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5585 - loss: 2.1133
Epoch 9/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5372 - loss: 1.6751
Epoch 10/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5506 - loss: 1.4392
Epoch 11/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5543 - loss: 1.3023
Epoch 12/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5607 - loss: 1.2633
Epoch 13/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accu

In [ ]:
y_pred_off = np.argmax(model_sarc.predict(x_test_off), axis=1)

accuracy = accuracy_score(y_test_off, y_pred_off)
f1_off = f1_score(y_test_off, y_pred_off, average="macro")  
print("Sarcasm")
print(f"Predicted Labels: {y_pred_off}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Macro F1 Score: {f1_off:.4f}")

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Sarcasm
Predicted Labels: [1 1 1 ... 1 1 1]
Accuracy: 0.5976
Macro F1 Score: 0.3740
